## Loading Data

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers
from tensorflow import data as tf_data
from tensorflow.keras.callbacks import EarlyStopping

c:\Users\dinuk\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# tfds.disable_progress_bar()
train_60,val_20,test_20=tfds.load("oxford_iiit_pet",split=["train+test[:20%]","test[20%:60%]","test[60%:]"],as_supervised=True)

resizing=layers.Resizing(224,224)
train_60=train_60.map(lambda x,y:(resizing(x),y))
val_20=val_20.map(lambda x,y:(resizing(x),y))
test_20=test_20.map(lambda x,y:(resizing(x),y))

batch_size = 64

train_60 = train_60.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
val_20 = val_20.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test_20 = test_20.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [2]:
train,test=tfds.load("oxford_iiit_pet",split=["train","test"],as_supervised=True)

resizing=layers.Resizing(224,224)
train=train.map(lambda x,y:(resizing(x),y))
test=test.map(lambda x,y:(resizing(x),y))

batch_size = 64

train = train.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()
test = test.batch(batch_size).prefetch(tf_data.AUTOTUNE).cache()

In [5]:
F10_train=tfds.load("oxford_iiit_pet",split=[f"train[:{r}%]+train[{r+10}%:]" for r in range(0,100,10)],as_supervised=True)
F10_val=tfds.load("oxford_iiit_pet",split=[f"train[{r}%:{r+10}%]" for r in range(0,100,10)],as_supervised=True)

## Model

In [3]:
base_model=keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3),pooling="avg")

In [4]:
base_model.trainable=False

In [8]:
inputs=keras.Input(shape=(224,224,3))   
preprocessing=tf.keras.applications.resnet.preprocess_input(inputs)
x=base_model(preprocessing, training=False)
x=layers.Dense(512,activation="relu")(x)
x=layers.Dropout(0.4)(x)    
outputs=layers.Dense(37)(x)

model=keras.Model(inputs,outputs)

loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizers=keras.optimizers.Adam(learning_rate=0.0002,decay=0.00001)

model.compile(optimizer=optimizers,loss=loss_fn,metrics=[keras.metrics.SparseTopKCategoricalAccuracy( k=3, name="accuracy")])

In [9]:
model.summary(show_trainable=True) 

Model: "model_1"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         Y          
                                                                            
 tf.__operators__.getitem_1   (None, 224, 224, 3)      0         Y          
 (SlicingOpLambda)                                                          
                                                                            
 tf.nn.bias_add_1 (TFOpLambd  (None, 224, 224, 3)      0         Y          
 a)                                                                         
                                                                            
 resnet50 (Functional)       (None, 2048)              23587712  N          
                                                                            
 dense_2 (Dense)             (None, 512)               1049

In [10]:
callback=EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)
model.fit(train,epochs=5,validation_data=test,callbacks=[callback],verbose=1)

Epoch 1/5


58/58 [==============================] - 27s 397ms/step - loss: 2.2180 - accuracy: 0.6348 - val_loss: 0.9146 - val_accuracy: 0.9469
Epoch 2/5
58/58 [==============================] - 22s 374ms/step - loss: 0.7641 - accuracy: 0.9432 - val_loss: 0.5770 - val_accuracy: 0.9703
Epoch 3/5
58/58 [==============================] - 21s 368ms/step - loss: 0.4686 - accuracy: 0.9793 - val_loss: 0.4668 - val_accuracy: 0.9768
Epoch 4/5
58/58 [==============================] - 21s 372ms/step - loss: 0.3404 - accuracy: 0.9897 - val_loss: 0.4243 - val_accuracy: 0.9779
Epoch 5/5
58/58 [==============================] - 22s 379ms/step - loss: 0.2615 - accuracy: 0.9921 - val_loss: 0.3922 - val_accuracy: 0.9782


In [ ]:
model.evaluate(test_20,verbose=2)   